# Imports

In [3]:
import imageio
from PIL import Image
import numpy as np
from sklearn.neighbors import KernelDensity
import cv2
from scipy.signal import argrelextrema
import time
import math
import scipy.cluster.hierarchy as hcluster
import pandas as pd
import os
import zipfile
from matplotlib.pyplot import plot
from sklearn.cluster import DBSCAN
import torch

# Download data from Kaggle (this is several gigs of video)
uncomment if data undownloaded

In [4]:
# !pip install -q kaggle
# !mkdir ~/.kaggle
# #!cp kaggle.json ~/.kaggle/kaggle.json
# !echo '{"username":"keeganmoore8","key":"667f611262e0c6c107060b7f8dab7fc0"}' > ~/.kaggle/kaggle.json
# !kaggle competitions download -c nfl-impact-detection
# !mkdir data
# !mv nfl-impact-detection.zip data
# zip_ref = zipfile.ZipFile('data/nfl-impact-detection.zip', 'r')
# zip_ref.extractall('data')
# zip_ref.close()

# YOLOv8 download
first is downloading

In [5]:
# !git clone https://github.com/RizwanMunawar/yolov8-object-tracking.git
# !cd yolov8-object-tracking
# !pip install ultralytics==8.0.0
# !pip install filterpy

# Darknet YOLOv3 and v4 Player detection (obsolete)
No longer used, v8 better

In [6]:
net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')
net = cv2.dnn.readNetFromDarknet('yolov4.cfg', 'yolov4.weights')
output_layer_names = net.getLayerNames()
output_layer_names = [output_layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

In [7]:
# def find_players(image):
#     box_start_time = time.time()
#     buffer = 100
#     im_arr = np.array(image)
    
#     height, width, channels = im_arr.shape
#     blob = cv2.dnn.blobFromImage(im_arr[buffer:len(im_arr)-buffer,buffer:len(im_arr[0])-buffer] , 1 / 255.0, (320, 320), swapRB=True, crop=False)
#     cv2.imwrite("test.png", im_arr[buffer:len(im_arr)-buffer,buffer:len(im_arr[0])-buffer])
#     net.setInput(blob)
#     layer_outputs = net.forward(output_layer_names)

#     boxes = []
#     confidences = []
#     class_ids = []

#     for output in layer_outputs:
#         for detection in output:
#             scores = detection[5:]
#             class_id = np.argmax(scores)
#             confidence = scores[class_id]

#             if confidence > 0.1 and detection[2] < .3 and detection[3] < .3:
#                 center_x = int(detection[0] * (width-2*buffer))+buffer
#                 center_y = int(detection[1] * (height-2*buffer))+buffer
#                 w = int(detection[2] * width)
#                 h = int(detection[3] * height)

#                 x = int(center_x - w / 2)
#                 y = int(center_y - h / 2)

#                 boxes.append([x, y, w, h])
#                 confidences.append(float(confidence))
#                 class_ids.append(class_id)

#     indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.1, 0.2)
#     return boxes, indices, confidences, class_ids, time.time()-box_start_time

## Dated labeller with v3/v4 (obsolete)

In [8]:
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# for name in ['Sideline']:
#     filename = f'data/train/57584_002674_Endzone.mp4'
#     vid = imageio.get_reader(filename,  'ffmpeg')
#     out = cv2.VideoWriter(f'{name}_out.mp4',fourcc, vid.get_meta_data()['fps'], (len(vid.get_data(0)[0]),len(vid.get_data(0))))
#     line_tot_time = 0
#     box_tot_time = 0
#     time_tot = 0
#     count = 0
#     start_time = time.time()
#     for frame in range(int(vid.get_meta_data()['duration']*vid.get_meta_data()['fps'])):
#         count+=1
#         frame_time_start = time.time()
#         image = Image.fromarray(vid.get_data(frame))
#         lines, line_time = find_lines(image)
#         line_tot_time+=line_time
#         boxes, indices, confidences, class_ids, box_time = find_players(image)
#         box_tot_time += box_time
#         im_arr = np.array(image)
#         for line in lines:
#             cv2.line(im_arr,(line[0][0],line[0][1]),(line[1][0],line[1][1]),(255,0,0),2)
#         for i in indices:
#             box = boxes[i]
#             x, y, w, h = box
#             label = str(class_ids[i])
#             confidence = confidences[i]
#             color = (0, 255, 0)
#             cv2.rectangle(im_arr, (x, y), (x+w, y+h), color, 2)
#         out.write(im_arr)
#         time_tot+= time.time()-frame_time_start
#     out.release()
#     print(f'total time: {time.time()-start_time} seconds')
#     print(f'average frame time: {time_tot/count} seconds')
#     print(f'line time: {line_tot_time} seconds')
#     print(f'box time: {box_tot_time} seconds')
#     print(f'drawing time: {time_tot - box_tot_time - line_tot_time} seconds')

# Field Line Detector

In [10]:
def find_lines(image):
        line_start_time = time.time()
        im_arr = np.array(image)
        gray = image.convert("L")
        gray_arr = np.array(gray)
        edges = cv2.Canny(gray_arr,100,150,apertureSize=3)  #Canny to get edge map
        lines_list =[]
        lines = cv2.HoughLinesP(     #use cv2's hough lines to get possible lines
                    edges, # Input edge image
                    1, # Distance resolution in pixels
                    np.pi/180, # Angle resolution in radians
                    threshold=200, # Min number of votes for valid line
                    minLineLength=min(image.height, image.width)/6, # Min allowed length of line
                    maxLineGap=25 # Max allowed gap between line for joining them
                    )
        for points in lines:
              # Extracted points nested in the list
            x1,y1,x2,y2=points[0]
            if x2!=x1:     #need special case for vertical lines, infinte slope
                slope = (y2-y1)/(x2-x1)
                if slope == 0:
                    lines_list.append((slope, int(image.width/2), int(y1))) #slope, center_x, center_y
                else:
                    x_int = (slope*x1-y1)/slope
                    y_int = y1-x1*slope
                    if x_int > 0 and x_int < image.width:
                        lower_point = (x_int, 0)
                    else:
                        lower_point = (0, y_int)
                    wid_int = ((image.height-y2)/slope)+x2
                    hei_int = slope*(image.width-x2)+y2
                    if hei_int > 0 and hei_int < image.height:
                        upper_point = (image.width, hei_int)
                    else:
                        upper_point = (wid_int, image.height)
                    lines_list.append((slope, int((upper_point[0]+lower_point[0])/2), int((upper_point[1]+lower_point[1])/2))) #slope, center_x, center_y
            else:
                lines_list.append((999999, int(x1), int(image.height/2))) #slope, center_x, center_y
        draw_lines = []
        thresh = 30
        if len(lines_list)>1:    #non-maximal suppression for lines
            clusters = hcluster.fclusterdata(lines_list, thresh, criterion="distance")
            df = pd.DataFrame(lines_list, index=clusters).sort_index()
            real_lines = []
            for i in range(1, max(df.index)+1):
                curr = df[df.index==i].median(axis=0)
                real_lines.append(list(curr))

            df2 = pd.DataFrame(real_lines)
            df2.at[df2[2]==0,2]=1
            df2.insert(3, 'angle', abs(np.arctan(df2[2]/(df2[1]-(df2[1]-(df2[2]/df2[0])))))) #get angles of lines from x-axis
            
            if len(df2['angle']) != 0:
                #cluster angles, then take biggest cluster as the field lines as opposed to other lines
                clustering = DBSCAN(eps=.15, min_samples=2).fit(list(zip(df2['angle'], np.zeros(len(df2['angle'])))))
                best_ind = -1
                best_score = -1
                for i in range(len(set(clustering.labels_))):
                    if np.count_nonzero(clustering.labels_ == i) > best_score:
                        best_score = np.count_nonzero(clustering.labels_ == i)
                        best_ind = i
                df2 = df2[clustering.labels_[df2.index]==best_ind]
                if len(df2.index) < 2:
                    print("BAD FRAME")
                for x, curr in df2.iterrows():  
                    if int(curr[0]) == 999999:
                        draw_lines.append(((int(curr[1]), 0), (int(curr[1]), image.height)))
                    elif curr[0] == 0:
                        draw_lines.append(((0, int(curr[2])), (image.width, int(curr[2]))))
                    else:
                        draw_lines.append(((int(curr[1]-(curr[2]/curr[0])),0),(int((image.height-curr[2])/curr[0]+curr[1]),image.height)))

                        
        return draw_lines, time.time()-line_start_time

# Main Function for Line+Player Detection (with v8)
creates a video

In [1]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
for name in ['Sideline']:
    filename = f'data/57584_002674_Sideline.mp4'
    vid = imageio.get_reader(filename,  'ffmpeg')
    out = cv2.VideoWriter(f'{name}_out.mp4',fourcc, vid.get_meta_data()['fps'], (len(vid.get_data(0)[0]),len(vid.get_data(0))))
    line_tot_time = 0
    box_tot_time = 0
    time_tot = 0
    count = 0
    start_time = time.time()
    #v8 player detection, puts bounding boxes into a torch file:
    !python3 yolov8-object-tracking/yolo/v8/detect/detect_and_trk.py model=yolov8m.pt source='data/57584_002674_Sideline.mp4'
    t = torch.load('preds.pt')
    for frame in range(int(vid.get_meta_data()['duration']*vid.get_meta_data()['fps'])):
        count+=1
        frame_time_start = time.time()
        image = Image.fromarray(vid.get_data(frame))
        lines, line_time = find_lines(image) #get lines in frame
        line_tot_time+=line_time
        boxes = t[frame][0][:, :4]
        im_arr = np.array(image)
        #draw lines and boxes onto video:
        for line in lines:
            cv2.line(im_arr,(line[0][0],line[0][1]),(line[1][0],line[1][1]),(0,0,255),2)
        for box in boxes:
            x, y, x2, y2 = box
            x = int(x)
            y = int(y)
            x2=int(x2)
            y2=int(y2)
            color = (255, 0, 0)
            cv2.rectangle(im_arr, (x, y), (x2, y2), color, 2)
        out.write(im_arr)
        time_tot+= time.time()-frame_time_start
    out.release()


# Helmet Detection

In [2]:
!python3 yolov5/detect.py --weights best.pt \
                  --source 'Sideline_out.mp4' \
                  --img 720 \
                  --hide-labels \
                  --save-txt \
                  --save-conf \
                  --project 'out'